### INTENT AND EMOTION DETECTION USING DistilBERT

In [16]:
# Import required libraries
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

In [17]:
# Define intent keywords
intent_keywords = {
    "venting": ["frustrated", "angry", "fed up", "tired", "exhausted"],
    "seeking_advice": ["what should I do", "any suggestions", "advice", "recommend"],
    "gratitude": ["thank you", "appreciate", "grateful", "thanks a lot"],
    "general_chat": ["how are you", "what's up", "hello", "hi"],
}
# Define personas
persona_styles = {
    "companion": lambda msg: f"I'm right here with you. {msg} We can face this together.",
    "coach": lambda msg: f"Alright, let's break it down. {msg} You’ve got what it takes.",
    "listener": lambda msg: f"I'm listening... {msg} Take your time, I'm here for it.",
}

In [18]:
# Get user input
user_input = input("Enter a sentence: ")
chosen_persona = input("Choose a persona (companion, coach, listener): ")

Enter a sentence: I broke up with my girlfriend.
Choose a persona (companion, coach, listener): companion


In [19]:
# Intent detection
def detect_intent(text):
  text = text.lower()
  for intent, keywords in intent_keywords.items():
    if any(keyword in text for keyword in keywords):
      return intent
  return "general_chat"
intent = detect_intent(user_input)

In [20]:
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('nateraw/bert-base-uncased-emotion')
model = AutoModelForSequenceClassification.from_pretrained('nateraw/bert-base-uncased-emotion')

# Preprocess input
inputs = tokenizer(user_input, return_tensors="pt")

# Run through the model
with torch.no_grad():
  outputs = model(**inputs)

# Get emotion probabilities
probs = F.softmax(outputs.logits, dim=1)
predicted_emotion = torch.argmax(probs, dim=1).item()
emotion_label = model.config.id2label[predicted_emotion]

In [24]:
# Respond based on emotion
base_response = {
    "sadness": "I'm really sorry you're feeling this way. Want to talk more about what's weighing you down?",
    "joy": "That's great to hear! Tell me more about what made you feel this way.",
    "love": "That sounds really heartwarming. Want to share more?",
    "anger": "That sounds frustrating. I'm here if you need to vent.",
    "fear": "It’s okay to feel scared sometimes. Would it help to break it down together?",
    "surprise": "Oh wow! That must have been unexpected. Tell me what happened.",
}.get(emotion_label, "I'm here for you. Want to tell me more?")

In [26]:
# Output chatbot reply
final_response = persona_styles[chosen_persona](base_response)
print(f"Detected emotion: {emotion_label}")
print(f"Intent: {intent}")
print(f"Chatbot: {final_response}")

Detected emotion: sadness
Intent: general_chat
Chatbot: I'm right here with you. I'm really sorry you're feeling this way. Want to talk more about what's weighing you down? We can face this together.
